In [89]:
import os
import pandas as pd
import gmaps
import googlemaps
import numpy as np
import json
from ipywidgets.embed import embed_minimal_html
import time

In [90]:
os.path.exists('../data/Location_BaDinh.xlsx')

True

In [91]:
store_locs = '../data/Location_BaDinh.xlsx'

In [92]:
df = pd.read_excel(store_locs)

In [93]:
df.head()

,Unnamed: 0,branch_no,branch_name,corporate_id,corporate_name,branch_address_line_1,Unnamed: 0.1,correct_lat,correct_lng,vcm_id
0,72,917,AK67_C0752_HN_8C11 Giảng Võ,AK67,VinMart+,"số 8 - c11 giảng võ, phường giảng võ, quận ba ...",63,21.027593,105.823790,2536
1,110,3594,AK67_3465_VM+ HNI 671 Hoàng Hoa Thám,AK67,VinMart+,"tầng 1, công trình dịch vụ và nhà ở cao tầng t...",94,21.046719,105.807253,3465
2,203,1116,AK67_C0838_HN_109 Trần Huy Liệu,AK67,VinMart+,"số 109 trần huy liệu, tổ dân phố 7a, phường gi...",174,21.027164,105.822591,2752
3,305,5598,VM+ HNI 637 Kim Mã,AK67,VinMart+,"637 kim mã, p. ngọc khánh, q. ba đình, hà nội",271,21.029171,105.807325,4825
4,358,155,AK67_C0515_HN_K2 Vĩnh Phúc,AK67,VinMart+,"nhà k2, khu tt 7,2ha, phường vĩnh phúc, quận b...",2322,21.042752,105.808253,2067


In [94]:
len(df)

34

In [95]:
gmaps.configure(api_key=os.getenv('GOOGLE_API_KEY'))
GG_CLIENT = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))
assert GG_CLIENT is not None, "Please check Google API KEY"

In [96]:
warehouse = (21.078295, 105.966889)

In [97]:
lats = [warehouse[0]] + list(df.correct_lat)
lngs = [warehouse[1]] + list(df.correct_lng)

In [98]:
covered_lat = np.min(lats) + (np.max(lats) - np.min(lats)) / 2
covered_lng = np.min(lngs) + (np.max(lngs) - np.min(lngs)) / 2
covered_lat, covered_lng

(21.051025250000002, 105.8869532)

In [99]:
center_lat = np.mean(lats)
center_lng = np.mean(lngs)
center_lat, center_lng

(21.038952902857147, 105.82771623999999)

In [100]:
layout = {
    'width': "{}px".format(1200),
    'height': "{}px".format(800),}
# Init figure

fig = gmaps.figure(center=(covered_lat, covered_lng), zoom_level=13, layout=layout)

# Plot depot with maker layer
locations = list(zip(df.correct_lat, df.correct_lng))
# labels = [str(i) if i != self.tsptw_data.data['depot'] else 'Depot' for i in range(len(self.tsptw_data.optimal_tsp))]
# info_box_contents = []
# for idx in self.tsptw_data.optimal_tsp:
#     address = self.tsptw_data.addresses[idx - 1]
#     tw = self.tsptw_data.data['time_window'][idx - 1]
#     time_marker = self.tsptw_data.time_marker 
#     opened_tw = time.strftime("%d/%m/%y %H:%M:%S", time.gmtime(time_marker + tw[0]))
#     closed_tw = time.strftime("%d/%m/%y %H:%M:%S", time.gmtime(time_marker + tw[1]))
#     content = """
#     <div class="poi-info-window gm-style"> 
#     <div jstcache="126" class="title full-width" jsan="7.title,7.full-width">{}</div>
#     <div jstcache="127" jsinstance="1" class="address-line full-width" jsan="7.address-line,7.full-width">Open: {}</div>
#     <div jstcache="127" jsinstance="1" class="address-line full-width" jsan="7.address-line,7.full-width">Close: {}</div>
#     </div>""".format(address, opened_tw, closed_tw)
#     info_box_contents.append(content)
symbol_layer = gmaps.symbol_layer(locations, fill_color='red', stroke_color='red', scale=3)
marker_layer = gmaps.marker_layer([warehouse])
fig.add_layer(symbol_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='800px', width='1200px'))

# Matrix

In [265]:
def get_google_distance_matrix(origins, destinations, mode='driving'):
    if len(origins) * len(destinations) < 100:
        return GG_CLIENT.distance_matrix(origins=origins, destinations=destinations, mode=mode)
    else:
        dist = None
        dura = None
        for i, origin in enumerate(origins):
            gg_matrix = GG_CLIENT.distance_matrix(origins=origin, destinations=destinations, mode=mode)
            distance_matrix, duration_matrix = get_utility_matrices(gg_matrix)
            if i == 0:
                dist = distance_matrix
                dura = duration_matrix
            else:
                dist = np.concatenate((dist, distance_matrix), axis=0)
                dura = np.concatenate((dura, duration_matrix), axis=0)
        return dist, dura

In [270]:
gg_dist_mat = get_google_distance_matrix(tmp_locs[:], tmp_locs[:])

In [243]:
def parse_into_np_matrix(rows, size, dtype='distance'):
    np_matrix = np.zeros(size)
    for i, row in enumerate(rows):
        for j in range(len(row['elements'])):
            np_matrix[i][j] = row['elements'][j][dtype]['value']
    return np_matrix

In [244]:
def get_utility_matrices(google_matrix):
    if google_matrix['status'] == 'OK':
        size = (len(google_matrix['origin_addresses']), len(google_matrix['destination_addresses']))
        distance_matrix = parse_into_np_matrix(google_matrix['rows'], size)
        duration_matrix = parse_into_np_matrix(google_matrix['rows'], size, dtype='duration')
        return (distance_matrix, duration_matrix)
    return (None, None)

In [245]:
tmp_locs = list(zip(lats, lngs))
tmp_locs[:10]

[(21.078295, 105.966889),
 (21.027593, 105.82379),
 (21.0467195, 105.8072527),
 (21.0271642, 105.8225908),
 (21.0291707, 105.8073247),
 (21.0427522, 105.808253),
 (21.0237555, 105.8171405),
 (21.038003, 105.810641),
 (21.030377, 105.824848),
 (21.0333895, 105.8170932)]

In [246]:
gg_dist_mat = get_google_distance_matrix(tmp_locs[:11], tmp_locs[:11])
# distance_matrix, duration_matrix = get_utility_matrices(gg_dist_mat)

(21.078295, 105.966889)
[[    0. 33562. 31727. 33476. 34324. 32832. 25743. 32433. 26084. 34371.
  24914.]]
(21.027593, 105.82379)
[[32941.     0.  4276.   164.  3738.  3523.   939.  2470.   622.  1339.
   2844.]]
(21.0467195, 105.8072527)
[[25349.  3978.     0.  3902.  4395.  1146.  4125.  2243.  3829.  2557.
   3000.]]
(21.0271642, 105.8225908)
[[32952.   164.  4351.     0.  3813.  3598.   950.  2545.   766.  1414.
   2918.]]
(21.0291707, 105.8073247)
[[33644.  2060.  3161.  1984.     0.  2408.  2208.  2009.  1911.  1365.
   3698.]]
(21.0427522, 105.808253)
[[36509.  4503.  1106.  4427.  3428.     0.  4651.  2120.  4355.  2556.
   2999.]]
(21.0237555, 105.8171405)
[[32433.  2679.  4108.  1974.  3652.  3356.     0.  2520.  2530.  1995.
   4328.]]
(21.038003, 105.810641)
[[37228.  2630.  1867.  2554.  2152.  1114.  2778.     0.  2481.  1209.
   2788.]]
(21.030377, 105.824848)
[[24363.   622.  4127.   687.  3590.  3375.  1604.  2321.     0.  1191.
   2179.]]
(21.0333895, 105.8170932)
[[3

In [248]:
gg_dist_mat[0]

array([[ 1.38338381e-322,  4.67338574e-310,  6.93872450e-310,
         6.93876791e-310,  6.93876442e-310,  0.00000000e+000,
         4.67338573e-310,  0.00000000e+000,  0.00000000e+000,
         5.96769772e+054,  0.00000000e+000],
       [ 4.24399159e-314,  6.17116420e-114,  5.34742939e-312,
         2.54639495e-312,  1.59089138e-321,  3.64983276e-312,
         2.56761491e-312,  4.44659081e-322,  2.88848595e+174,
         9.75020870e+199,  4.76990878e+180],
       [ 2.10934367e-110,  6.01347002e-154,  5.04758700e+223,
         8.82880751e+199,  1.28515803e+248,  4.54522494e+223,
         1.07548461e+272,  3.50636654e+151,  4.25525961e-119,
         1.05773753e-153,  1.81148219e-152],
       [ 1.99416198e+174,  4.98131536e+151,  2.92572348e-014,
         3.75504822e+199,  6.93630429e+218,  4.38883662e+228,
         3.99277570e+252,  2.02791796e+267,  4.61964705e+281,
         4.27255707e+180,  4.67338344e-310],
       [ 6.93872510e-310,  4.67338344e-310,  0.00000000e+000,
         0.000

In [217]:
gg_dist_mat

{'destination_addresses': ['Số 1 Đường Hữu Nghị, Phù Chẩn, Từ Sơn, Bắc Ninh, Vietnam',
  'F8 C11 Giảng Võ, Ba Đình, Hà Nội, Vietnam',
  '671 Hoàng Hoa Thám, Vĩnh Phú, Ba Đình, Hà Nội, Vietnam',
  '109 Trần Huy Liệu, Giảng Võ, Ba Đình, Hà Nội, Vietnam',
  '645 Kim Mã, Ngọc Khánh, Ba Đình, Hà Nội, Vietnam',
  'Ngõ 462 - Bưởi, Vĩnh Phú, Ba Đình, Hà Nội, Vietnam',
  '575 Đường La Thành, Thành Công, Ba Đình, Hà Nội, Vietnam',
  '443 Đội Cấn, Vĩnh Phú, Ba Đình, Hà Nội, Vietnam',
  '60 Ngõ Núi Trúc, Kim Mã, Ba Đình, Hà Nội, Vietnam',
  '69 Vạn Bảo, Liễu Giai, Ba Đình, Hà Nội, Vietnam'],
 'origin_addresses': ['Số 1 Đường Hữu Nghị, Phù Chẩn, Từ Sơn, Bắc Ninh, Vietnam',
  'F8 C11 Giảng Võ, Ba Đình, Hà Nội, Vietnam',
  '671 Hoàng Hoa Thám, Vĩnh Phú, Ba Đình, Hà Nội, Vietnam',
  '109 Trần Huy Liệu, Giảng Võ, Ba Đình, Hà Nội, Vietnam',
  '645 Kim Mã, Ngọc Khánh, Ba Đình, Hà Nội, Vietnam',
  'Ngõ 462 - Bưởi, Vĩnh Phú, Ba Đình, Hà Nội, Vietnam',
  '575 Đường La Thành, Thành Công, Ba Đình, Hà Nội, Vie

In [109]:
distance_matrix

array([[    0., 33562., 31727., 33476., 34324., 32832., 25743., 32433.,
        26084., 34371.],
       [32941.,     0.,  4276.,   164.,  3738.,  3523.,   939.,  2470.,
          622.,  1339.],
       [25349.,  3978.,     0.,  3902.,  4395.,  1146.,  4125.,  2243.,
         3829.,  2557.],
       [32952.,   164.,  4351.,     0.,  3813.,  3598.,   950.,  2545.,
          766.,  1414.],
       [33644.,  2060.,  3161.,  1984.,     0.,  2408.,  2208.,  2009.,
         1911.,  1365.],
       [36509.,  4503.,  1106.,  4427.,  3428.,     0.,  4651.,  2120.,
         4355.,  2556.],
       [32433.,  2679.,  4108.,  1974.,  3652.,  3356.,     0.,  2520.,
         2530.,  1995.],
       [37228.,  2630.,  1867.,  2554.,  2152.,  1114.,  2778.,     0.,
         2481.,  1209.],
       [24363.,   622.,  4127.,   687.,  3590.,  3375.,  1604.,  2321.,
            0.,  1191.],
       [33905.,  1593.,  2796.,  1517.,  2912.,  2043.,  1741.,  1207.,
         1445.,     0.]])

In [110]:
duration_matrix

array([[   0., 3163., 3109., 3133., 3176., 3240., 3048., 3121., 3053.,
        3106.],
       [2937.,    0.,  828.,  109.,  752.,  768.,  235.,  551.,  199.,
         292.],
       [2877.,  896.,    0.,  943.,  845.,  317.,  880.,  474.,  850.,
         538.],
       [2976.,   87.,  849.,    0.,  773.,  789.,  274.,  572.,  235.,
         313.],
       [2909.,  493.,  549.,  540.,    0.,  489.,  477.,  374.,  447.,
         312.],
       [2844.,  870.,  245.,  918.,  466.,    0.,  854.,  353.,  824.,
         570.],
       [2853.,  574.,  735.,  439.,  707.,  675.,    0.,  475.,  528.,
         335.],
       [2875.,  632.,  326.,  680.,  372.,  267.,  616.,    0.,  586.,
         274.],
       [2725.,  206.,  789.,  286.,  713.,  729.,  345.,  512.,    0.,
         253.],
       [2942.,  391.,  554.,  439.,  585.,  494.,  375.,  293.,  345.,
           0.]])

# Convert to JSON format

In [111]:
import os
import gmaps
import googlemaps
import argparse
import numpy as np
import json
import time
import geopy

In [112]:
geocoder = geopy.geocoders.GoogleV3(api_key=os.getenv('GOOGLE_API_KEY'))

In [116]:
geocoder.reverse(tmp_locs[0], exactly_one=True)

Location(Số 1 Đường Hữu Nghị, Phù Chẩn, Từ Sơn, Bắc Ninh, Vietnam, (21.0782946, 105.967074, 0.0))

In [128]:
geocode = GG_CLIENT.reverse_geocode(tmp_locs[0])

In [132]:
# Save data into json
json_data = {}
json_data['destination'] = []
depot = 0
for latlng in tmp_locs:
    geocode_res = GG_CLIENT.reverse_geocode(latlng)
    json_data['destination'].append(geocode_res[0])
# json_data['time_window'] = time_window
json_data['depot'] = depot
json.dump(json_data, open('./badinh_tsp.json', 'w'))

In [193]:
# Load data from json
json_data = json.load(open('./badinh_tsp.json', 'r'))
json_data

{'destination': [{'address_components': [{'long_name': 'Số 1',
     'short_name': 'Số 1',
     'types': ['street_number']},
    {'long_name': 'Đường Hữu Nghị',
     'short_name': 'Đường Hữu Nghị',
     'types': ['route']},
    {'long_name': 'Thị xã Từ Sơn',
     'short_name': 'Tx. Từ Sơn',
     'types': ['locality', 'political']},
    {'long_name': 'Bắc Ninh',
     'short_name': 'Bắc Ninh',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Vietnam',
     'short_name': 'VN',
     'types': ['country', 'political']}],
   'formatted_address': 'Số 1 Đường Hữu Nghị, Phù Chẩn, Từ Sơn, Bắc Ninh, Vietnam',
   'geometry': {'location': {'lat': 21.0782946, 'lng': 105.967074},
    'location_type': 'ROOFTOP',
    'viewport': {'northeast': {'lat': 21.0796435802915,
      'lng': 105.9684229802915},
     'southwest': {'lat': 21.0769456197085, 'lng': 105.9657250197085}}},
   'place_id': 'ChIJ6V7zeTWpNTERvCUI7JBMHG0',
   'plus_code': {'compound_code': '3XH8+8R Từ Sơn, Bac Ni

# Examine

In [204]:
os.listdir('../data/tw_data')

['data_9_1.txt',
 'data_8_4.txt',
 'data_16_2.txt',
 'data_15_5.txt',
 'data_20_3.txt',
 'data_12_13.txt',
 'data_12_17.txt',
 'data_17_1.txt',
 'data_11_5.txt',
 'data_7_1.txt',
 'data_19_5.txt',
 'data_17_2.txt',
 'data_14_4.txt',
 'data_9_4.txt',
 'data_13_1.txt',
 'data_14_5.txt',
 'data_20_1.txt',
 'data_12_16.txt',
 'data_13_2.txt',
 'data_12_1.txt',
 'data_20_2.txt',
 'data_12_10.txt',
 'data_12_6.txt',
 'data_14_1.txt',
 'data_12_9.txt',
 'data_7_5.txt',
 'data_17_4.txt',
 'data_17_5.txt',
 'data_19_4.txt',
 'data_10_5.txt',
 'data_9_5.txt',
 'data_12_14.txt',
 'data_15_1.txt',
 'data_12_20.txt',
 'data_18_3.txt',
 'data_14_2.txt',
 'data_12_15.txt',
 'data_13_3.txt',
 'data_16_3.txt',
 'data_20_4.txt',
 'data_12_3.txt',
 'data_9_2.txt',
 'data_15_4.txt',
 'data_9_3.txt',
 'data_12_8.txt',
 'data_12_11.txt',
 'data_19_3.txt',
 'data_19_2.txt',
 'data_7_3.txt',
 'data_17_3.txt',
 'data_13_4.txt',
 'data_7_4.txt',
 'data_8_3.txt',
 'data_11_1.txt',
 'data_16_4.txt',
 'data_18_4.t

In [205]:
sample = json.load(open('../data/tw_data/data_15_2.txt', 'r'))

In [215]:
def generate_tw(json_data):
    time_window = list(zip(json_data['ET_i'], json_data['LT_i']))
    time_window = time_window[-1:] + time_window[1:-1]
    return json_data['Location'], time_window

In [216]:
generate_tw(sample)

([3, 4, 5, 6, 14, 16, 17, 19, 24, 25, 27, 28, 29, 30, 32],
 [(0, 10000),
  (90, 270),
  (180, 360),
  (150, 330),
  (210, 390),
  (90, 180),
  (120, 210),
  (210, 300),
  (210, 390),
  (30, 210),
  (210, 390),
  (30, 210),
  (0, 90),
  (210, 300),
  (180, 360),
  (90, 180)])